# Introduction

In [37]:
!pip install -U --quiet pandas gretel-client


In [38]:
!gretel configure


Gretel.ai COPYRIGHT Notice


The Gretel CLI and Python SDK, installed through the "gretel-client"
package or other mechanism is free and open source software under
the Apache 2.0 License.

When using the CLI or SDK, you may launch "Gretel Worker(s)"
that are hosted in your local environment as containers. These
workers are launched automatically when running commands that create
models or process data records.

The "Gretel Worker" and all code within it is copyrighted and an
extension of the Gretel Service and licensed under the Gretel.ai
Terms of Service.  These terms can be found at https://gretel.ai/terms
section G paragraph 2.



Endpoint [https://api.gretel.cloud]: 
Artifact Endpoint [cloud]: 
Default Runner (cloud, local, hybrid) [cloud]: 
Gretel API Key [grtu6836****]: 
Default Project [none]: 
Using endpoint https://api.gretel.cloud
Logged in as ericphamhung@gmail.com ✅
INFO: Configuration written to /root/.gretel/config.json. Done.
{
    "endpoint": "https://api.gretel.cloud"

In [8]:
!gretel whoami

{
    "email": "ericphamhung@gmail.com",
    "config": {
        "endpoint": "https://api.gretel.cloud",
        "artifact_endpoint": "cloud",
        "api_key": "grtu6836****",
        "default_project_name": null,
        "default_runner": "cloud",
        "preview_features": "disabled"
    }
}


# Main

In [39]:
import pandas as pd
from gretel_client import Gretel



In [40]:
df = pd.read_csv('transactions_merged.csv')
df.head()

,customer_id,transaction_date,transaction_amount,transaction_type,transaction_category,starting_amount,account_balance
0,CUST_001,2022-01-01,100.0,credit,rent,1000.0,1100.0
1,CUST_001,2022-01-01,50.0,debit,groceries,1000.0,1050.0
2,CUST_001,2022-01-02,75.0,credit,salary,1000.0,1125.0
3,CUST_001,2022-01-02,25.0,debit,entertainment,1000.0,1100.0
4,CUST_001,2022-01-03,150.0,credit,investment,1000.0,1250.0


In [41]:
df[df['customer_id']=='CUST_055']

,customer_id,transaction_date,transaction_amount,transaction_type,transaction_category,starting_amount,account_balance
281,CUST_055,2022-08-05,15000.0,debit,Purchase,750.0,-14250.0
874,CUST_055,2022-02-06,60.0,debit,subscription,750.0,-14310.0


In [42]:
df[df['starting_amount'].isna()].customer_id.unique()

array([], dtype=object)

In [44]:
train_df = df
# train_df = df.sample(frac=0.8, random_state=42)
# test_df = df.drop(train_df.index)

In [45]:
train_df.drop(columns=['account_balance'], inplace=True)
train_df

,customer_id,transaction_date,transaction_amount,transaction_type,transaction_category,starting_amount
0,CUST_001,2022-01-01,100.0,credit,rent,1000.0
1,CUST_001,2022-01-01,50.0,debit,groceries,1000.0
2,CUST_001,2022-01-02,75.0,credit,salary,1000.0
3,CUST_001,2022-01-02,25.0,debit,entertainment,1000.0
4,CUST_001,2022-01-03,150.0,credit,investment,1000.0
...,...,...,...,...,...,...
995,CUST_048,2023-01-17,1210.0,credit,salary,0.0
996,CUST_049,2023-01-18,1000.0,debit,groceries,0.0
997,CUST_049,2023-01-18,1230.0,credit,salary,0.0
998,CUST_050,2023-01-19,1020.0,debit,dining,0.0


In [46]:
from gretel_client.projects.models import read_model_config

config_dict = read_model_config("synthetics/default")

config_dict

{'schema_version': '1.0',
 'name': 'tabular-lstm',
 'models': [{'synthetics': {'data_source': '__tmp__',
    'params': {'epochs': 'auto',
     'vocab_size': 'auto',
     'learning_rate': 'auto',
     'batch_size': 'auto',
     'rnn_units': 'auto'},
    'generate': {'num_records': 5000},
    'privacy_filters': {'outliers': 'auto', 'similarity': 'auto'}}}]}

In [47]:
import pandas as pd

from gretel_client import create_or_get_unique_project
from gretel_client import poll

train_df.columns = train_df.columns.astype(str)
proj = create_or_get_unique_project(name="gretel-assignment")


# Train model on "real" data
model = proj.create_model_obj(model_config="synthetics/tabular-lstm", data_source=train_df)

model.submit_cloud()


Model(id=6751f8ff0de2667a8fff96dc, project=proj_2pidXfwDmSfJeUhxqsVlaLv2mGm)

In [48]:
poll(model) #view progress

INFO: Starting poller


{
    "uid": "6751f8ff0de2667a8fff96dc",
    "guid": "model_2poHFyGpScdh6HLLNoZMpKYFKXP",
    "model_name": "tabular-lstm",
    "model_key": "",
    "runner_mode": "cloud",
    "user_id": "674f25bb04ad3047687b5568",
    "user_guid": "user_2piDxsBzv9Bui42s1SGlF8FOQ2F",
    "billing_domain": "47b326e1cf4946efafc6298a7030aeba.gretel",
    "billing_domain_guid": "domain_2plpw9z74e3cBK2BtTVyERBueqt",
    "project_id": "674f5706002447fa52833677",
    "project_guid": "proj_2pidXfwDmSfJeUhxqsVlaLv2mGm",
    "cluster_guid": null,
    "status_history": {
        "created": "2024-12-05T19:03:27.819895Z"
    },
    "last_modified": "2024-12-05T19:03:28.043895Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "annotations": null,
    "provenance": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/gretelai/synthetics@sha256:e09c09a1469d611d51739d4d8a8aa01c76b591ed1e05d

INFO: Status is created. Model creation has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated to begin model creation.
INFO: Status is active. A worker has started creating your model!
2024-12-05T19:04:05.609246Z  Analyzing input data and checking for auto-params...
2024-12-05T19:04:05.627197Z  Found 5 auto-params that were set based on input data.
{
    "epochs": 100,
    "batch_size": 64,
    "vocab_size": 20000,
    "rnn_units": 256,
    "learning_rate": 0.01
}
2024-12-05T19:04:05.627798Z  Using updated model configuration: 
{
    "schema_version": "1.0",
    "name": "tabular-lstm",
    "models": [
        {
            "synthetics": {
                "privacy_filters": {
                    "outliers": "auto",
                    "similarity": "auto",
                    "max_iterations": 10
                },
                "data_source": [
                    "gretel_f53171e906a14c96a1b71948aa3a1b27_dataframe-56093101-621d-4d9c-ab8e-a02e0c061ac5.csv"

In [49]:
# Generate Data
# NOTE: Model running params are provided as a dictionary
handler = model.create_record_handler_obj(params={"num_records": 1000})
handler.submit_cloud()

In [50]:
poll(handler)

INFO: Starting poller


{
    "uid": "6751fb48c80c76731f505df5",
    "guid": "model_run_2poIRQjo7zvp3jbRbMbIPvsig9i",
    "model_name": null,
    "model_key": "",
    "runner_mode": "cloud",
    "user_id": "674f25bb04ad3047687b5568",
    "user_guid": "user_2piDxsBzv9Bui42s1SGlF8FOQ2F",
    "billing_domain": "47b326e1cf4946efafc6298a7030aeba.gretel",
    "billing_domain_guid": "domain_2plpw9z74e3cBK2BtTVyERBueqt",
    "project_id": "674f5706002447fa52833677",
    "project_guid": "proj_2pidXfwDmSfJeUhxqsVlaLv2mGm",
    "cluster_guid": null,
    "status_history": {
        "created": "2024-12-05T19:13:12.739000Z"
    },
    "last_modified": "2024-12-05T19:13:12.842000Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "annotations": null,
    "provenance": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/gretelai/synthetics@sha256:e09c09a1469d611d51739d4d8a8aa01c76b591ed1e05deea50f

INFO: Status is created. A Record generation job has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated to begin generating synthetic records.
INFO: Status is active. A worker has started!
2024-12-05T19:13:29.007805Z  Loading model to worker
2024-12-05T19:13:47.496557Z  Checking for synthetic smart seeds
2024-12-05T19:13:47.496938Z  No smart seeds provided, will attempt generation without them
2024-12-05T19:13:47.497189Z  Loading model
2024-12-05T19:13:47.497557Z  Fallback model is available to use if needed.
2024-12-05T19:13:53.737434Z  LSTM model is available for generation.
2024-12-05T19:13:53.738228Z  Generating records...
{
    "num_records": 1000
}
2024-12-05T19:13:53.739549Z  Generation in progress
{
    "current_valid_count": 0,
    "current_invalid_count": 0,
    "new_valid_count": 0,
    "new_invalid_count": 0,
    "completion_percent": 0.0
}
2024-12-05T19:13:58.745663Z  Generation in progress
{
    "current_valid_count": 0,
    "current_invalid_co

In [51]:
handler.download_artifacts("more_syn_data/")

In [52]:
!gzip -d more_syn_data/data.gz


gzip: more_syn_data/data already exists; do you wish to overwrite (y or n)? y


In [53]:
synth_df = pd.read_csv("more_syn_data/data")

In [54]:
synth_df[synth_df['customer_id']=='CUST_001']

,customer_id,transaction_date,transaction_amount,transaction_type,transaction_category,starting_amount
74,CUST_001,2022-01-04,150.00,credit,loan,1000.0
75,CUST_001,2022-01-01,980.25,credit,deposit,1000.0
90,CUST_001,2022-01-01,80.00,credit,loan,1000.0
113,CUST_001,2022-01-02,25.00,credit,salary,1000.0
123,CUST_001,2022-01-02,300.00,debit,entertainment,1000.0
153,CUST_001,2022-01-04,30.00,credit,housing,1000.0
184,CUST_001,2022-01-02,25.00,credit,salary,1000.0
191,CUST_001,2022-01-01,80.00,credit,loan,1000.0
195,CUST_001,2022-01-03,300.00,credit,entertainment,1000.0
225,CUST_001,2022-01-01,50.00,credit,sports,1000.0


In [55]:
synth_df['customer_id'].value_counts()

,customer_id
CUST_004,51
CUST_009,47
CUST_014,44
CUST_013,44
CUST_006,42
...,...
CUST_060,1
CUST_052,1
CUST_069,1
CUST_071,1


In [56]:
train_df['customer_id'].value_counts()

,customer_id
CUST_002,43
CUST_001,41
CUST_005,38
CUST_009,38
CUST_006,37
...,...
CUST_059,1
CUST_058,1
CUST_057,1
CUST_056,1


In [59]:
# Define a function to compute the account balance
def compute_balance(row, balances):
    previous_balance = balances.get(row['customer_id'], row['starting_amount'])
    transaction_effect = row['transaction_amount'] if row['transaction_type'] == "credit" else -row['transaction_amount']
    current_balance = previous_balance + transaction_effect
    balances[row['customer_id']] = current_balance
    return current_balance

# Compute the account balance for each transaction
balances = {}
synth_df['account_balance'] = synth_df.apply(compute_balance, axis=1, balances=balances)


In [60]:
synth_df

,customer_id,transaction_date,transaction_amount,transaction_type,transaction_category,starting_amount,account_balance
0,CUST_012,2022-01-05,120.00,debit,rent,750.0,630.00
1,CUST_040,2023-01-09,500.00,debit,Food,0.0,-500.00
2,CUST_030,2022-02-01,1700.00,credit,housing,0.0,1700.00
3,CUST_012,2022-01-12,350.00,credit,investment,750.0,980.00
4,CUST_015,2022-02-26,590.00,debit,salary,450.0,-140.00
...,...,...,...,...,...,...,...
995,CUST_014,2022-03-17,25.00,debit,groceries,100.0,25930.00
996,CUST_012,2022-03-10,35.99,credit,bonus,750.0,30860.99
997,CUST_009,2022-01-28,1200.00,credit,rent,700.0,14785.99
998,CUST_004,2022-01-21,40.00,credit,rent,1200.0,6487.97


In [62]:
synth_df.to_csv('synthetic_data.csv', index=False)